In [1]:
# We found that certain columns are a sum of the columns that follow it, such as Power is a sum of Shot Power, Jumping,
# Stamina, Strength and Long Shots. Before we Train-Test, we can try drop the columns that sum up to the sumed column
# and also try dropping the sumed column to see which gives us a better output
# The '+' in the position columns indicate the potential improvement score in that position
# IR = International Reputation can be dropped
# Keeping ID to be able to search indivuals for now, but dropping when we come to train the model
# Could drop BP as we see that the first listed Position is the players best position
# We can drop Team & Contract as Team is a reflection of Club and we have a Contract column
# Convert Loan Date End to a Boolan as we have the end date in the contract
# For Joined, we will only keep the year as the exact date doesn't give us much value
# Converting Height to cm and excluding the metric name, as well as for Weight to kg
# Converting Value to actual numeric numnber e.g 6M to 6,000,000. Same with Wage and Release Clause
# Altering the Contract for the players on loan to reflect the start year(coming from Joined) and dropping the 'Loan' string
# so that all values in Contract will have a start year and an end year. Once this has been done, we can drop Joined column 
# https://fifauteam.com/fifa-21-attributes-guide/ useful to understanding the attributes
# Hits column is unknown for now, but it seems to have some correlation to the overal score.
# Position columns have the current value with '+' for their potential growth. We will split this column into 2, original column
# to reflect the current value and the new column will be the sum of the original column plus the potential value
# Keeping OVA as that is our Label

In [2]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import functools
from datetime import date
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import linear_model

In [3]:
fifa = pd.read_csv('./fifa21_train.csv')
fifa.columns = [i.lower().replace(' ', '_') for i in fifa.columns]

In [4]:
fifa.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)


In [5]:
lst = fifa.columns.unique()
fifa.dtypes

id              int64
name           object
age             int64
nationality    object
club           object
                ...  
cb             object
rcb            object
rb             object
gk             object
ova             int64
Length: 96, dtype: object

In [6]:
def minus(df):
    lst = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']
    for i in lst:
        df[[i, i + '_pot']] = df[i].str.split('+', expand=True)
        df[i + '_pot'] = df[i + '_pot'].astype('int') + df[i].astype('int')
        df[i] = df[i].astype('int')
    return df


In [7]:
minus(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,...,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",168lbs,Left,5,...,62,62,62,65,65,63,63,63,65,18
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,"6'1""",174lbs,Right,4,...,28,28,28,27,26,28,28,28,26,71
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,"5'6""",143lbs,Right,8,...,46,46,46,51,47,37,37,37,47,19
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,"6'2""",179lbs,Right,7,...,68,68,68,66,64,60,60,60,64,25


In [8]:
fifa.isna().sum()

id              0
name            0
age             0
nationality     0
club           18
               ..
lcb_pot         0
cb_pot          0
rcb_pot         0
rb_pot          0
gk_pot          0
Length: 123, dtype: int64

In [9]:
fifa.dropna(inplace=True) #We guess the NaN values are for players who are very young and don't have proper ratings yet, so we are dropping them.

In [10]:
fifa.drop_duplicates(inplace = True)

In [11]:
 def star(df):
    
    lst_str = ['w/f', 'sm', 'ir']
    
    for i in lst_str:
            
        df[[i]] = df[[i]].replace('★', '', regex=True)
        df[[i]] = df[[i]].astype('int')
        
    return df
    

In [12]:
star(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,...,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",154lbs,Right,0,...,47,47,47,48,46,45,45,45,46,21
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",168lbs,Left,5,...,62,62,62,65,65,63,63,63,65,18
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,"6'1""",174lbs,Right,4,...,28,28,28,27,26,28,28,28,26,71
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,"5'6""",143lbs,Right,8,...,46,46,46,51,47,37,37,37,47,19


In [13]:
fifa.dtypes

id              int64
name           object
age             int64
nationality    object
club           object
                ...  
lcb_pot         int32
cb_pot          int32
rcb_pot         int32
rb_pot          int32
gk_pot          int32
Length: 123, dtype: object

In [14]:
 def money(df):
    
    lst_mny = ['value', 'wage', 'release_clause', 'hits']
    
    for i in lst_mny:
            
        
        df[i] = df[i].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
        
    return df
    

In [15]:
money(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,...,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",154lbs,Right,0,...,47,47,47,48,46,45,45,45,46,21
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",168lbs,Left,5,...,62,62,62,65,65,63,63,63,65,18
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,"6'1""",174lbs,Right,4,...,28,28,28,27,26,28,28,28,26,71
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,"5'6""",143lbs,Right,8,...,46,46,46,51,47,37,37,37,47,19


In [16]:
def weight(df):
    
    lst_weight = ['weight']
    for i in lst_weight:
        df[i] = df[i].replace({'lbs':''}, regex=True).astype(int)
        df[i] = round(df[i]/2.205, 2) 
    
    return df


In [17]:
weight(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",73.02,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",72.11,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",60.77,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",68.93,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",68.03,Right,8,...,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",69.84,Right,0,...,47,47,47,48,46,45,45,45,46,21
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",76.19,Left,5,...,62,62,62,65,65,63,63,63,65,18
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,"6'1""",78.91,Right,4,...,28,28,28,27,26,28,28,28,26,71
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,"5'6""",64.85,Right,8,...,46,46,46,51,47,37,37,37,47,19


In [18]:
def height(df):
    lst_height =['height']
    for i in lst_height:
        df[i] = df[i].replace({'"':''}, regex=True)
        df[i] = df[i].apply(lambda x: 12*int(x.split("'")[0])+int(x.split("'")[1]))
        df[i] = df[i]*2.54
    return df


In [19]:
height(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,175.26,73.02,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,182.88,72.11,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,162.56,60.77,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,177.80,68.93,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,180.34,68.03,Right,8,...,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,170.18,69.84,Right,0,...,47,47,47,48,46,45,45,45,46,21
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,185.42,76.19,Left,5,...,62,62,62,65,65,63,63,63,65,18
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,185.42,78.91,Right,4,...,28,28,28,27,26,28,28,28,26,71
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,167.64,64.85,Right,8,...,46,46,46,51,47,37,37,37,47,19


In [20]:
fifa.head()

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,175.26,73.02,Right,1,...,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,182.88,72.11,Right,0,...,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,162.56,60.77,Right,0,...,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,177.80,68.93,Right,13,...,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,180.34,68.03,Right,8,...,66,66,66,66,65,63,63,63,65,17


In [21]:
fifa_cat = fifa.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1) #We decide to drop the nationality, club and name since we don't think those have any influence over the 'ova'.

In [22]:
fifa_num = fifa._get_numeric_data().drop(['id'], axis = 1) #We also drop the 'id' since it's unique for every player and would just create noise in un model.

In [23]:
X_num = fifa_num.drop(['ova'], axis = 1)

In [24]:
y = fifa_num['ova']

In [25]:
X_num.head()

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,26,175.26,73.02,1,525000.0,4000.0,801000.0,258,54,47,...,60,60,60,60,59,55,55,55,59,16
1,30,182.88,72.11,0,8500000.0,23000.0,0.0,365,66,79,...,55,55,55,59,55,50,50,50,55,20
2,33,162.56,60.77,0,9000000.0,49000.0,15300000.0,336,73,76,...,58,58,58,61,55,43,43,43,55,14
3,22,177.80,68.93,13,275000.0,4000.0,694000.0,242,44,42,...,60,60,60,58,59,60,60,60,59,16
4,23,180.34,68.03,8,725000.0,2000.0,1400000.0,249,49,37,...,66,66,66,66,65,63,63,63,65,17


In [26]:
transformer = MinMaxScaler().fit(X_num)
X_num_n = transformer.transform(X_num)
X_num_n = pd.DataFrame(X_num_n, columns=X_num.columns)
X_num_n.head()

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,0.370370,0.40,0.349317,0.038462,0.005833,0.007143,0.004811,0.546835,0.545455,0.478261,...,0.591549,0.591549,0.591549,0.619718,0.619718,0.521127,0.521127,0.521127,0.619718,0.0625
1,0.518519,0.55,0.333392,0.000000,0.094444,0.041071,0.000000,0.817722,0.681818,0.826087,...,0.521127,0.521127,0.521127,0.605634,0.563380,0.450704,0.450704,0.450704,0.563380,0.1125
2,0.629630,0.15,0.134932,0.000000,0.100000,0.087500,0.091892,0.744304,0.761364,0.793478,...,0.563380,0.563380,0.563380,0.633803,0.563380,0.352113,0.352113,0.352113,0.563380,0.0375
3,0.222222,0.45,0.277739,0.500000,0.003056,0.007143,0.004168,0.506329,0.431818,0.423913,...,0.591549,0.591549,0.591549,0.591549,0.619718,0.591549,0.591549,0.591549,0.619718,0.0625
4,0.259259,0.50,0.261988,0.307692,0.008056,0.003571,0.008408,0.524051,0.488636,0.369565,...,0.676056,0.676056,0.676056,0.704225,0.704225,0.633803,0.633803,0.633803,0.704225,0.0750


In [27]:
fifa_num_corr = fifa_num.corr()
fifa_num_corr.round(2)

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
age,1.00,0.09,0.25,-0.87,0.11,0.20,0.09,0.19,0.16,0.11,...,0.22,0.22,0.22,0.16,0.16,0.20,0.20,0.20,0.16,0.15
height,0.09,1.00,0.77,-0.05,-0.00,0.02,-0.00,-0.35,-0.49,-0.36,...,-0.18,-0.18,-0.18,-0.30,-0.24,-0.04,-0.04,-0.04,-0.24,0.35
weight,0.25,0.77,1.00,-0.22,0.04,0.07,0.03,-0.25,-0.39,-0.27,...,-0.12,-0.12,-0.12,-0.23,-0.18,0.01,0.01,0.01,-0.18,0.33
growth,-0.87,-0.05,-0.22,1.00,-0.19,-0.23,-0.17,-0.28,-0.25,-0.19,...,-0.30,-0.30,-0.30,-0.26,-0.24,-0.26,-0.26,-0.26,-0.24,-0.09
value,0.11,-0.00,0.04,-0.19,1.00,0.83,0.98,0.29,0.25,0.23,...,0.28,0.28,0.28,0.28,0.25,0.21,0.21,0.21,0.25,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lcb_pot,0.20,-0.04,0.01,-0.26,0.21,0.22,0.20,0.50,0.51,0.12,...,0.96,0.96,0.96,0.91,0.95,1.00,1.00,1.00,0.95,-0.57
cb_pot,0.20,-0.04,0.01,-0.26,0.21,0.22,0.20,0.50,0.51,0.12,...,0.96,0.96,0.96,0.91,0.95,1.00,1.00,1.00,0.95,-0.57
rcb_pot,0.20,-0.04,0.01,-0.26,0.21,0.22,0.20,0.50,0.51,0.12,...,0.96,0.96,0.96,0.91,0.95,1.00,1.00,1.00,0.95,-0.57
rb_pot,0.16,-0.24,-0.18,-0.24,0.25,0.25,0.24,0.65,0.71,0.29,...,0.98,0.98,0.98,0.99,1.00,0.95,0.95,0.95,1.00,-0.67


In [28]:
fifa_cat.head()

,bp,foot,a/w,d/w
0,CM,Right,High,Medium
1,ST,Right,High,Low
2,CAM,Right,High,Medium
3,CDM,Right,Medium,Medium
4,CDM,Right,Low,Medium


In [29]:
fifa_cat_hot = fifa_cat[['bp', 'foot']] #We decide to OneHot encode these two variables because they are categorical with no logical value order.
fifa_cat_hot.head()

,bp,foot
0,CM,Right
1,ST,Right
2,CAM,Right
3,CDM,Right
4,CDM,Right


In [30]:
fifa_cat_ord = fifa_cat[['a/w', 'd/w']] #We decide to Ordinal encode these variables because there is a logical order to the values they can take, and we want to give that some weight.
fifa_cat_ord.head()

,a/w,d/w
0,High,Medium
1,High,Low
2,High,Medium
3,Medium,Medium
4,Low,Medium


In [31]:
hot_encoder = OneHotEncoder(drop='first').fit(fifa_cat_hot)
hot_encoded = hot_encoder.transform(fifa_cat_hot).toarray()
hot_cols = hot_encoder.get_feature_names(input_features=fifa_cat_hot.columns)
hot_encoded = pd.DataFrame(hot_encoded, columns = [i.lower() for i in hot_cols])
hot_encoded.head()

,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
ord_encoder = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(fifa_cat_ord)

ord_encoded = pd.DataFrame((ord_encoder.transform(fifa_cat_ord)), 
                           columns = fifa_cat_ord.columns)
ord_encoded.head()


,a/w,d/w
0,2.0,1.0
1,2.0,0.0
2,2.0,1.0
3,1.0,1.0
4,0.0,1.0


In [33]:
transformer_ord = MinMaxScaler().fit(ord_encoded)
ord_encoded_n = transformer_ord.transform(ord_encoded)
ord_encoded_n = pd.DataFrame(ord_encoded_n, columns=fifa_cat_ord.columns)
ord_encoded_n.head()

,a/w,d/w
0,1.0,0.5
1,1.0,0.0
2,1.0,0.5
3,0.5,0.5
4,0.0,0.5


In [34]:
X = pd.concat([ord_encoded_n, hot_encoded, X_num_n],axis=1)
X.head()

,a/w,d/w,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,1.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.591549,0.591549,0.591549,0.619718,0.619718,0.521127,0.521127,0.521127,0.619718,0.0625
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.521127,0.521127,0.521127,0.605634,0.563380,0.450704,0.450704,0.450704,0.563380,0.1125
2,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.563380,0.563380,0.563380,0.633803,0.563380,0.352113,0.352113,0.352113,0.563380,0.0375
3,0.5,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.591549,0.591549,0.591549,0.591549,0.619718,0.591549,0.591549,0.591549,0.619718,0.0625
4,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.676056,0.676056,0.676056,0.704225,0.704225,0.633803,0.633803,0.633803,0.704225,0.0750


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [37]:
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9253995300697726

In [38]:
predictions_test = lm.predict(X_test)
r2 = r2_score(y_test, predictions_test)
mse = mean_squared_error(y_test, predictions_test)
mae = mean_absolute_error(y_test, predictions_test)
rmse = math.sqrt(mse)

In [39]:
print("r2 score is:", r2)
print("mse is:", mse)
print("mae is:",mae)
print("rmse is:",rmse)

r2 score is: 0.9254566251979788
mse is: 3.5911448216642663
mae is: 1.4648287168565433
rmse is: 1.8950316149511244


In [40]:
fifa_v = pd.read_csv('./fifa21_validate.csv')
fifa_v.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,...,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,...,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,...,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,...,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,...,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [41]:
fifa_v.columns = [i.lower().replace(' ', '_') for i in fifa_v.columns]
fifa_v.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)


In [42]:
minus(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,"6'5""",176lbs,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,"5'7""",146lbs,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,"5'11""",176lbs,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [43]:
star(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,"6'5""",176lbs,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,"5'7""",146lbs,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,"5'11""",176lbs,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [44]:
money(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,"6'5""",176lbs,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,"5'7""",146lbs,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,"5'11""",176lbs,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [45]:
weight(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",87.98,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",69.84,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",79.82,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",77.10,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",77.10,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",64.85,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,"6'5""",79.82,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,"5'7""",66.21,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,"5'11""",79.82,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [46]:
height(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,187.96,87.98,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,172.72,69.84,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,187.96,79.82,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,190.50,77.10,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,187.96,77.10,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,172.72,64.85,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,195.58,79.82,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,170.18,66.21,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,180.34,79.82,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [49]:
fifa_v.dropna(inplace=True)
fifa_v.drop_duplicates(inplace = True)

In [50]:
fifa_v_cat = fifa.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1)
fifa_v_num = fifa._get_numeric_data().drop(['id'], axis = 1)
X_num_v = fifa_v_num.drop(['ova'], axis = 1)
y_v = fifa_v_num['ova']

In [51]:
X_num_n_v = transformer.transform(X_num_v)
X_num_n_v = pd.DataFrame(X_num_n_v, columns=X_num_v.columns)

In [52]:
fifa_cat_hot_v = fifa_v_cat[['bp', 'foot']]
fifa_cat_ord_v = fifa_v_cat[['a/w', 'd/w']]

In [54]:
hot_encoded_v = hot_encoder.transform(fifa_cat_hot_v).toarray()
hot_cols_v = hot_encoder.get_feature_names(input_features=fifa_cat_hot_v.columns)
hot_encoded_v = pd.DataFrame(hot_encoded_v, columns = [i.lower() for i in hot_cols_v])

In [55]:
ord_encoder_v = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(fifa_cat_ord_v)

ord_encoded_v= pd.DataFrame((ord_encoder_v.transform(fifa_cat_ord_v)), 
                           columns = fifa_cat_ord_v.columns)

In [56]:
transformer_ord = MinMaxScaler().fit(ord_encoded_v)
ord_encoded_n_v = transformer_ord.transform(ord_encoded_v)
ord_encoded_n_v = pd.DataFrame(ord_encoded_n_v, columns=fifa_cat_ord_v.columns)

In [57]:
X_v = pd.concat([ord_encoded_n_v, hot_encoded_v, X_num_n_v],axis=1)

In [58]:
predictions_test_v = lm.predict(X_v)
r2_v = r2_score(y_v, predictions_test_v)
mse_v = mean_squared_error(y_v, predictions_test_v)
mae_v = mean_absolute_error(y_v, predictions_test_v)
rmse_v = math.sqrt(mse)

In [59]:
print("r2 score is:", r2_v)
print("mse is:", mse_v)
print("mae is:",mae_v)
print("rmse is:",rmse_v)

r2 score is: 0.9254112962789006
mse is: 3.4885504766841477
mae is: 1.448850948705379
rmse is: 1.8950316149511244


### Presenting results for our validation:

In [60]:
results_for_v = lm.predict(X_v)

fifa_v_results = fifa_v['ova']

Results = pd.concat([fifa_v_results,pd.Series(results_for_v)],axis=1)

Results.rename(columns = {'ova':'Target', 0:'Prediction'}, inplace = True)

Results.head()


,Target,Prediction
0,67.0,63.960251
1,68.0,76.697217
2,54.0,77.231298
3,55.0,61.436883
4,70.0,66.290963


# Bonus Round

### Combining functions with validating data:

In [59]:
test = pd.read_csv('./fifa21_validate.csv')

In [60]:
def clean_data(df):
    
    
    df.columns = [i.lower().replace(' ', '_') for i in df.columns]
    df = df.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)
    fifa.dropna(inplace=True)
        
    return df

In [61]:
clean_data(test)

In [62]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x) ##credits to https://mathieularose.com/function-composition-in-python


In [63]:
fifa_clean = compose(minus,money,star,height,weight)

In [64]:
fifa_clean(test)

,id,name,age,nationality,club,bp,height,weight,foot,growth,...,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,187.96,87.98,Right,7,...,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,172.72,69.84,Right,5,...,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,187.96,79.82,Right,17,...,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,190.50,77.10,Right,23,...,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,187.96,77.10,Right,5,...,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,172.72,64.85,Left,11,...,54,54,54,60,60,56,56,56,60,17
1995,241223,S. Mogi,21,Japan,Cerezo Osaka,GK,195.58,79.82,Right,9,...,25,25,25,22,22,24,24,24,22,60
1996,210930,Carles Gil,27,Spain,New England Revolution,RM,170.18,66.21,Left,0,...,62,62,62,64,59,49,49,49,59,20
1997,162993,J. Perch,34,England,Mansfield Town,CDM,180.34,79.82,Right,0,...,63,63,63,62,62,63,63,63,62,16


In [65]:
def l_model(df):
    
    df.dropna(inplace = True)
    
    df_cat = df.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1)
    df_num = df._get_numeric_data().drop(['id'], axis = 1)
    X_num = df_num.drop(['ova'], axis = 1)
    y = df_num['ova']
    
    transformer = MinMaxScaler().fit(X_num)
    X_num_n = transformer.transform(X_num)
    X_num_n = pd.DataFrame(X_num_n, columns=X_num.columns)
    X_num_n.head()
    
    df_cat_hot = df_cat[['bp', 'foot']]
    df_cat_ord = df_cat[['a/w', 'd/w']]
    
    hot_encoder = OneHotEncoder(drop='first').fit(df_cat_hot)
    hot_encoded = hot_encoder.transform(df_cat_hot).toarray()
    hot_cols = hot_encoder.get_feature_names(input_features=df_cat_hot.columns)
    hot_encoded = pd.DataFrame(hot_encoded, columns = [i.lower() for i in hot_cols])
    
    ord_encoder = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(df_cat_ord)

    ord_encoded = pd.DataFrame((ord_encoder.transform(df_cat_ord)), columns = df_cat_ord.columns)
    
    transformer_ord = MinMaxScaler().fit(ord_encoded)
    ord_encoded_n = transformer_ord.transform(ord_encoded)
    ord_encoded_n = pd.DataFrame(ord_encoded_n, columns=df_cat_ord.columns)
    
    X = pd.concat([ord_encoded_n, hot_encoded, X_num_n],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    lm = linear_model.LinearRegression()
    lm.fit(X_train,y_train)
    
    predictions = lm.predict(X_train)
    r2_score(y_train, predictions)
    
    predictions_test = lm.predict(X_test)
    r2 = r2_score(y_test, predictions_test)
    mse = mean_squared_error(y_test, predictions_test)
    mae = mean_absolute_error(y_test, predictions_test)
    rmse = math.sqrt(mse)
    
    print("r2 score is:", r2)
    print("mse is:", mse)
    print("mae is:",mae)
    print("rmse is:",rmse)
    
    results_for_v = lm.predict(X)

    df_results = df['ova']

    Results = pd.concat([df_results,pd.Series(results_for_v)],axis=1)

    Results.rename(columns = {'ova':'Target', 0:'Prediction'}, inplace = True)

     
    
    return Results


In [66]:
l_model(test)

r2 score is: 0.9282878219373285
mse is: 3.0916487256153493
mae is: 1.3881630189722056
rmse is: 1.758308484201606


,Target,Prediction
0,67.0,65.475230
1,68.0,65.801298
2,54.0,53.533605
3,55.0,55.936043
4,70.0,69.403724
...,...,...
1994,60.0,NaN
1995,59.0,NaN
1996,76.0,NaN
1997,63.0,NaN
